# Menjalankan Eksperimen

Anda dapat menggunakan SDK Azure Machine Learning untuk menjalankan eksperimen kode yang mencatat metrik dan menghasilkan output. Ini adalah inti dari sebagian besar operasi pembelajaran mesin di Azure Machine Learning.

## Menghubungkan ke ruang kerja Anda

Semua eksperimen dan sumber daya terkait dikelola dalam ruang kerja Azure Machine Learning Anda. Dalam kebanyakan kasus, Anda harus menyimpan konfigurasi ruang kerja dalam file konfigurasi JSON. Hal ini memudahkan untuk terhubung kembali tanpa perlu mengingat detail seperti ID langganan Azure Anda. Anda dapat mengunduh file konfigurasi JSON dari panel untuk ruang kerja Anda di portal Microsoft Azure, tetapi jika Anda menggunakan Instans Komputasi dalam ruang kerja, file konfigurasi telah diunduh ke folder akar.

Kode di bawah ini menggunakan file konfigurasi untuk terhubung ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Menjalankan eksperimen

Salah satu tugas paling mendasar yang perlu dilakukan ilmuwan data adalah membuat dan menjalankan eksperimen yang memproses dan menganalisis data. Dalam latihan ini, Anda akan mempelajari cara menggunakan *eksperimen* Azure Machine Learning untuk menjalankan kode Python dan mencatat nilai yang diekstrak dari data. Dalam kasus ini, Anda akan menggunakan himpunan data sederhana yang berisi detail pasien yang telah diperiksa untuk diabetes. Anda akan menjalankan eksperimen untuk menjelajahi data, mengekstrak statistik, visualisasi, dan sampel data. Sebagian besar kode yang akan Anda gunakan adalah Python yang cukup umum, seperti yang mungkin Anda jalankan dalam proses eksplorasi data apa pun. Tetapi, dengan penambahan beberapa baris, kode tersebut menggunakan *eksperimen* Azure Machine Learning untuk mencatat detail eksekusi.

In [ ]:
from azureml.core import Experiment
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-diabetes")

# Start logging data from the experiment, obtaining a reference to the experiment run
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the data from a local file
data = pd.read_csv('data/diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Plot and log the count of diabetic vs non-diabetic patients
diabetic_counts = data['Diabetic'].value_counts()
fig = plt.figure(figsize=(6,6))
ax = fig.gca()    
diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name='label distribution', plot=fig)

# log distinct pregnancy counts
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

# Log summary statistics for numeric columns
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat=keys[index], value = values[index])
        
# Save a sample of the data and upload it to the experiment output
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

# Complete the run
run.complete()

## Melihat detail eksekusi

Di Jupyter Notebooks, Anda dapat menggunakan widget **RunDetails** untuk melihat visualisasi detail eksekusi.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

### Melihat detail selengkapnya di studio Azure Machine Learning

Perhatikan bahwa widget **RunDetails** menyertakan tautan ke **melihat detail eksekusi** di studio Azure Machine Learning. Klik ini untuk membuka tab browser baru dengan detail eksekusi (Anda juga dapat membuka [Studio Azure Machine Learning](https://ml.azure.com) dan menemukan eksekusi di halaman **Eksperimen**). Saat melihat eksekusi di studio Azure Machine Learning, perhatikan hal berikut:

- Tab **Detail** berisi properti umum dari eksekusi eksperimen.
- Tab **Metrik** memungkinkan Anda memilih metrik yang dicatat dan melihatnya sebagai tabel atau diagram.
- Tab **Gambar** memungkinkan Anda memilih dan melihat gambar atau plot apa pun yang dicatat dalam eksperimen (dalam kasus ini, plot *Distribusi Label*)
- Tab **Eksekusi Turunan** mencantumkan semua eksekusi turunan (dalam eksperimen ini tidak ada eksekusi turunan).
- Tab **Output + Log** menampilkan file output atau log yang dihasilkan oleh eksperimen.
- Tab **Snapshot** berisi semua file dalam folder tempat kode eksperimen dijalankan (dalam kasus ini, semuanya berada di folder yang sama dengan buku catatan ini).
- Tab **Penjelasan** digunakan untuk menampilkan penjelasan model yang dihasilkan oleh eksperimen (dalam kasus ini tidak ada penjelasan model).
Tab **Keadilan** digunakan untuk memvisualisasikan perbedaan performa prediktif yang membantu Anda mengevaluasi keadilan model pembelajaran mesin (dalam kasus ini, tidak ada keadilan model pembelajaran mesin).

### Mengambil detail eksperimen menggunakan SDK

Variabel **eksekusi** dalam kode yang Anda jalankan sebelumnya adalah instans dari objek **Eksekusi**, yang merupakan referensi ke masing-masing eksekusi dari eksperimen di Azure Machine Learning. Anda dapat menggunakan referensi ini untuk mendapatkan informasi tentang eksekusi dan output eksekusi:

In [ ]:
import json

# Get logged metrics
print("Metrics:")
metrics = run.get_metrics()
for metric_name in metrics:
    print(metric_name, ":", metrics[metric_name])

# Get output files
print("\nFiles:")
files = run.get_file_names()
for file in files:
    print(file)

Anda dapat mengunduh file yang dihasilkan oleh eksperimen, baik satu per satu dengan menggunakan metode **download_file**, atau dengan menggunakan metode **download_files** untuk mengambil beberapa file. Kode berikut mengunduh semua file di folder **output** eksekusi:

In [ ]:
import os

download_folder = 'downloaded-files'

# Download files in the "outputs" folder
run.download_files(prefix='outputs', output_directory=download_folder)

# Verify the files have been downloaded
for root, directories, filenames in os.walk(download_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

Jika Anda perlu memecahkan masalah eksekusi eksperimen, Anda dapat menggunakan metode **get_details** untuk mengambil detail dasar tentang eksekusi tersebut, atau Anda dapat menggunakan metode **get_details_with_logs** untuk mengambil detail eksekusi serta konten file log yang dihasilkan selama eksekusi:

In [ ]:
run.get_details_with_logs()

Perhatikan bahwa detail mencakup informasi tentang target komputasi tempat eksperimen dijalankan, tanggal dan waktu dimulai dan berakhirnya eksperimen. Selain itu, karena buku catatan yang berisi kode eksperimen (yang ini) terdapat di repositori Git yang dikloning, detail tentang repositori, cabang, dan status dicatat dalam riwayat eksekusi.

Dalam kasus ini, perhatikan bahwa entri **logFiles** dalam detail menunjukkan bahwa tidak ada file log yang dibuat. Hal tersebut biasa terjadi untuk eksperimen sebaris seperti yang Anda jalankan, tetapi segalanya menjadi lebih menarik saat Anda menjalankan skrip sebagai eksperimen; yang akan kita lihat selanjutnya.

## Menjalankan skrip eksperimen

Pada contoh sebelumnya, Anda menjalankan sebaris eksperimen di buku catatan ini. Solusi yang lebih fleksibel adalah membuat skrip terpisah untuk eksperimen, dan menyimpannya di folder bersama dengan file lain yang diperlukan, lalu menggunakan Azure Machine Learning untuk menjalankan eksperimen berdasarkan skrip di folder.

Pertama, mari kita buat folder untuk file eksperimen, dan salin data ke dalamnya:

In [ ]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'diabetes-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

Sekarang kita akan membuat skrip Python yang berisi kode untuk eksperimen kita, dan menyimpannya di folder eksperimen.

> **Catatan**: menjalankan sel berikut hanya *membuat* file skrip - tidak menjalankannya!

In [ ]:
%%writefile $folder_name/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Count and log the label counts
diabetic_counts = data['Diabetic'].value_counts()
print(diabetic_counts)
for k, v in diabetic_counts.items():
    run.log('Label:' + str(k), v)
      
# Save a sample of the data in the outputs folder (which gets uploaded automatically)
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

Kode ini adalah versi sederhana dari kode sebaris yang digunakan sebelumnya. Tetapi, perhatikan hal berikut:
- Kode menggunakan metode `Run.get_context()` untuk mengambil konteks eksekusi eksperimen saat skrip dijalankan.
- Kode memuat data diabetes dari folder tempat skrip berada.
- Kode membuat folder bernama **outputs** dan menulis file sampel ke dalamnya - folder ini secara otomatis diunggah ke eksekusi eksperimen

Sekarang Anda hampir siap untuk menjalankan eksperimen. Untuk menjalankan skrip, Anda harus membuat **ScriptRunConfig** yang mengidentifikasi file skrip Python untuk dijalankan dalam eksperimen, lalu menjalankan eksperimen berdasarkan skrip tersebut.

> **Catatan**: ScriptRunConfig juga menentukan target komputasi dan lingkungan Python. Dalam kasus ini, lingkungan Python ditentukan untuk menyertakan beberapa paket Conda dan pip, tetapi target komputasi dihilangkan; sehingga komputasi lokal default akan digunakan.

Sel berikut mengonfigurasi dan mengirimkan eksperimen berbasis skrip.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_experiment.py',
                                environment=env,
                                docker_runtime_config=DockerConfiguration(use_docker=True))

# submit the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

Seperti sebelumnya, Anda dapat menggunakan widget atau tautan ke eksperimen di [Studio Azure Machine Learning](https://ml.azure.com) untuk melihat output yang dihasilkan oleh eksperimen, dan Anda juga dapat menulis kode untuk mengambil metrik dan file yang dihasilkan kode:

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Perhatikan bahwa kali ini, eksekusi menghasilkan beberapa file log. Anda dapat melihat ini di widget, atau Anda dapat menggunakan metode **get_details_with_logs** seperti yang kita lakukan sebelumnya, hanya saja kali ini output akan menyertakan data log.

In [ ]:
run.get_details_with_logs()

Meskipun Anda dapat melihat detail log pada output di atas, biasanya lebih mudah untuk mengunduh file log dan melihatnya di editor teks.

In [ ]:
import os

log_folder = 'downloaded-logs'

# Download all files
run.get_all_logs(destination=log_folder)

# Verify the files have been downloaded
for root, directories, filenames in os.walk(log_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

## Melihat riwayat eksekusi eksperimen

Sekarang setelah Anda menjalankan eksperimen yang sama beberapa kali, Anda dapat melihat riwayat di [Studio Azure Machine Learning](https://ml.azure.com) dan menjelajahi setiap eksekusi yang dicatat. Atau Anda dapat mengambil eksperimen berdasarkan nama dari ruang kerja dan mengulangi eksekusi ruang kerja menggunakan SDK:

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['mslearn-diabetes']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

## Menggunakan MLflow

MLflow adalah platform sumber terbuka untuk mengelola proses pembelajaran mesin. MLflow biasanya (tetapi tidak secara eksklusif) digunakan di lingkungan Databricks untuk mengoordinasikan eksperimen dan melacak metrik. Dalam eksperimen Azure Machine Learning, Anda dapat menggunakan MLflow untuk melacak metrik sebagai alternatif dari fungsionalitas log native.

Untuk memanfaatkan kemampuan ini, Anda memerlukan paket **azureml-mlflow**, jadi pastikan paket tersebut sudah diinstal.

In [ ]:
pip show azureml-mlflow

### Menggunakan MLflow dengan eksperimen sebaris

Untuk menggunakan MLflow guna melacak metrik eksperimen sebaris, Anda harus mengatur MLflow *URI pelacakan* ke ruang kerja tempat eksperimen dijalankan. Hal ini memungkinkan Anda menggunakan metode pelacakan **mlflow** untuk mencatat data ke eksekusi eksperimen.

In [ ]:
from azureml.core import Experiment
import pandas as pd
import mlflow

# Set the MLflow tracking URI to the workspace
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
mlflow.set_experiment(experiment.name)

# start the MLflow experiment
with mlflow.start_run():
    
    print("Starting experiment:", experiment.name)
    
    # Load data
    data = pd.read_csv('data/diabetes.csv')

    # Count the rows and log the result
    row_count = (len(data))
    mlflow.log_metric('observations', row_count)
    print("Run complete")

Sekarang mari kita lihat metrik yang dicatat selama eksekusi

In [ ]:
# Get the latest run of the experiment
run = list(experiment.get_runs())[0]

# Get logged metrics
print("\nMetrics:")
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
    
# Get a link to the experiment in Azure ML studio   
experiment_url = experiment.get_portal_url()
print('See details at', experiment_url)

Setelah menjalankan kode di atas, Anda dapat menggunakan tautan yang ditampilkan untuk melihat eksperimen di studio Azure Machine Learning. Kemudian pilih eksperimen terbaru dan lihat tab **Metrik** untuk melihat metrik yang dicatat.

### Menggunakan MLflow dalam skrip eksperimen

Anda juga dapat menggunakan MLflow untuk melacak metrik dalam skrip eksperimen.

Jalankan dua sel berikut untuk membuat folder dan skrip untuk eksperimen yang menggunakan MLflow.

In [ ]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'mlflow-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

In [ ]:
%%writefile $folder_name/mlflow_diabetes.py
from azureml.core import Run
import pandas as pd
import mlflow


# start the MLflow experiment
with mlflow.start_run():
       
    # Load data
    data = pd.read_csv('diabetes.csv')

    # Count the rows and log the result
    row_count = (len(data))
    print('observations:', row_count)
    mlflow.log_metric('observations', row_count)

Saat Anda menggunakan pelacakan MLflow dalam skrip eksperimen Azure Machine Learning, URI pelacakan MLflow diatur secara otomatis saat Anda memulai eksekusi eksperimen. Tetapi, lingkungan tempat skrip akan dijalankan harus menyertakan paket **mlflow** yang diperlukan.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails


# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_mlflow = ScriptRunConfig(source_directory=experiment_folder,
                                script='mlflow_diabetes.py',
                                environment=env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) 

# submit the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
run = experiment.submit(config=script_mlflow)
RunDetails(run).show()
run.wait_for_completion()

Seperti biasa, Anda bisa mendapatkan metrik yang dicatat dari eksekusi eksperimen setelah selesai.

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))

> **Informasi Selengkapnya**: Untuk mengetahui selengkapnya tentang menjalankan eksperimen, lihat [topik ini](https://docs.microsoft.com/azure/machine-learning/how-to-manage-runs) di dokumentasi Azure Machine Learning. Untuk detail tentang cara mencatat metrik dalam eksekusi, lihat [topik ini](https://docs.microsoft.com/azure/machine-learning/how-to-track-experiments). Untuk informasi selengkapnya tentang mengintegrasikan eksperimen Azure Machine Learning dengan MLflow, lihat [topik ini](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-mlflow).